### Convert Audio file into text

In [14]:
# Load the audio file
import librosa
file_path = "input.wav"
# read audio file and convert it into numpy array
audio, rate = librosa.load(file_path, sr=16000)
from transformers import pipeline
pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")
# Transcribe the audio
transcription = pipe(audio)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [15]:
transcription['text']

'BEST BEER IN BREMEN'

### Feed the Transcription into LLM and obtain the results

In [2]:
# import packages
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import chromadb
from datasets import load_dataset
from gpt4all import GPT4All
from pathlib import Path

import torch
from TTS.api import TTS
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

/home/jayanth/anaconda3/envs/LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# Path to your local text file
file_path = "data_bremen.txt"

# Initialize the loader
loader = TextLoader(file_path)

# Load the document
docs = loader.load()


splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=30)

chunked_docs = splitter.split_documents(docs)


db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})


RuntimeError: Error loading data_bremen.txt

In [4]:
model_path="gpt4all"
model_name = 'Meta-Llama-3-8B-Instruct.Q4_0.gguf'
model = GPT4All(model_name=model_name,model_path=model_path)

FileNotFoundError: Model directory does not exist: PosixPath('model')

In [18]:
user_input =  transcription['text']
# Retrieve relevant documents based on the user's input
results = retriever.invoke(user_input)

# Extract the context from the retrieved documents
context = results[0].page_content if results else ""

# Combine user input and context into a structured prompt
prompt = f"User input: {user_input}\nContext:{context} \nPlease provide a detailed, coherent response in paragraph format based on the above information .Do not add addresses and references in the response"

# Generate a response using the model
response = model.generate(prompt,max_tokens=120,temp=0.2)

# Print the generated response
print(response)

.

As you wander through Bremen's charming streets, you can't help but notice the vibrant craft beer scene that has taken over the city. With numerous breweries popping up left and right, it's no wonder locals and visitors alike are eager to try their hand at sampling some of the best brews around. Amongst the many excellent options available, one stands out as a clear favorite among Bremen's beer enthusiasts: the aptly named "Bremener Lager" from Brauerei Fiege.

This delightful lager is brewed with precision and care by master brewers who have honed


In [19]:
response

'.\n\nAs you wander through Bremen\'s charming streets, you can\'t help but notice the vibrant craft beer scene that has taken over the city. With numerous breweries popping up left and right, it\'s no wonder locals and visitors alike are eager to try their hand at sampling some of the best brews around. Amongst the many excellent options available, one stands out as a clear favorite among Bremen\'s beer enthusiasts: the aptly named "Bremener Lager" from Brauerei Fiege.\n\nThis delightful lager is brewed with precision and care by master brewers who have honed'

In [20]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Init TTS
tts = TTS("tts_models/en/jenny/jenny").to(device)
tts.tts_to_file(text=response, speaker_wav="output.wav", file_path="output.wav")

 > tts_models/en/jenny/jenny is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:48000
 | > resample:False
 | > num_mels:100
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:2048
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:512
 | > win_length:2048
 > Text splitted to sentences.
['.', "As you wander through Bremen's charming streets, you can't help but notice the vibrant craft beer scene that has taken over the city.", "With n

'output.wav'